In [1]:
import cassandra
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json
import pandas as pd
import fiddler as fdl

In [2]:
cassandra.__version__

'3.28.0'

In [4]:
# This secure connect bundle is autogenerated when you donwload your SCB, 
# if yours is different update the file name below
cloud_config= {
  'secure_connect_bundle': 'datastax_auth/secure-connect-fiddlerai.zip'
}

# This token json file is autogenerated when you donwload your token, 
# if yours is different update the file name below
with open("datastax_auth/danny@fiddler.ai-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

In [5]:
session.set_keyspace('fiddlerai')

In [6]:
def pandas_factory(colnames, rows):
    return pd.DataFrame(rows, columns=colnames)

session.row_factory = pandas_factory
session.default_fetch_size = None

In [33]:
rows = session.execute('SELECT * FROM fiddler_doc_snippets_openai')
    
df_docs = rows._current_rows
df_docs

,row_id,attributes_blob,body_blob,metadata_s,vector
0,992f28c79fa541a6a0a58b4eafc86921,None,"---\ntitle: ""client.delete_webhook""\nslug: ""cl...",None,"[-0.0026160315610468388, -0.026444554328918457..."
1,d03e2f3fe2514a4c8a52229fb6b13f48,None,"slug: ""publishing-events-with-complex-data-for...",None,"[-0.02673957124352455, 0.006959426216781139, -..."
2,3ea3314d6ebd4395ab423dab14f9b6a2,None,"---\ntitle: ""Class-Imbalanced Data""\nslug: ""cl...",None,"[-0.015294374898076057, 0.006210180465131998, ..."
3,08798909cdb94a6992590d78eda3ab0f,None,"slug: ""data-drift"" e-Monitor_DriftAnaly.png ""M...",None,"[-0.031133130192756653, -0.007197968661785126,..."
4,8fd0b121342142a397bd602bd8a78da9,None,"---\ntitle: ""Useful Queries for Root Cause Ana...",None,"[0.008876001462340355, 0.027711128816008568, -..."
...,...,...,...,...,...
310,e2ff7d1601934e59aa6e2383b936bbaa,None,"---\ntitle: ""Data Drift""\nslug: ""data-drift""\n...",None,"[-0.034394051879644394, -0.009222312830388546,..."
311,9e11d5e6645640608effb1001ea230b0,None,"slug: ""point-explanations"" prediction differe...",None,"[-0.020307065919041634, 0.021863754838705063, ..."
312,b50f8505df554a32bd58a530d057b61b,None,"---\ntitle: ""fdl.RowDataSource""\nslug: ""fdlrow...",None,"[-0.02189440280199051, 0.025069715455174446, -..."
313,f36dee885b324c7490ce16b13a8bc90d,None,"slug: ""class-imbalance-monitoring-example"" ler...",None,"[-0.0004906170070171356, -0.0091997180134058, ..."


In [32]:
rows = session.execute('SELECT row_id, session_id, source_docs, response, question, comment, feedback, ts FROM fiddler_chatbot_history')
    
df = rows._current_rows
df

,row_id,session_id,source_docs,response,question,comment,feedback,ts
0,feee7fdb-db14-46b8-a149-3502a1329df1,None,"Document: ---\ntitle: ""About the Fiddler Cl...",Fiddler is a company that provides an AI Obser...,Who is Fiddler?,None,NaN,2023-10-12 13:46:36.185
1,c710094f-c235-4967-a576-6e94b8f61c70,b36feb54-7464-482c-ad6c-71edd91d1585,Document: Re-uploading in Fiddler essential...,"To upload multiple events, you can use the `cl...",How can I upload multiple events?,None,NaN,2023-10-12 14:51:34.137
2,98950cca-94cf-4fee-9ab7-0c16c8ada4c7,b36feb54-7464-482c-ad6c-71edd91d1585,"Document: ---\ntitle: ""About Datasets""\nslu...","Currently, there isn''t a way for users to dir...",Can I delete datasets associated with a model?,None,NaN,2023-10-12 14:50:32.511
3,c5c14f05-df8c-402c-9fd3-3af8f6999231,fefc3bb2-724a-43ae-953f-651351a5a5f2,"Document: ---\ntitle: ""Baselines""\nslug: ""f...",There is no specific mention of the maximum nu...,How many of them can I have?,None,1.0,2023-10-12 13:44:06.765
4,8420f35f-14d8-4783-a3c6-1caffc62209d,d5a4f9af-63b2-4199-b392-17dcfdff4f93,"Document: ---\ntitle: ""System Architecture""...",I could not find an answer.\nJoin our [Slack c...,What’s on Fiddler roadmap?,None,NaN,2023-10-12 14:53:27.549
5,6a4d0f9b-5564-4340-a54a-6c10da12e179,d5a4f9af-63b2-4199-b392-17dcfdff4f93,"Document: ---\ntitle: ""Dashboards""\nslug: ""...",Dashboards and PSI (Project Structure Interfac...,What’s the difference between Dashboards and PSI?,None,NaN,2023-10-12 14:53:59.878
6,dfff104b-81ee-4e63-b864-a87b25aa2150,None,"Document: ---\ntitle: ""About the Fiddler Cl...",The purpose of Fiddler is to provide a platfor...,How is Fiddler?,None,NaN,2023-10-12 13:47:05.875
7,1e21aade-7d3a-4ba5-9c30-3bcd59e55fe5,fefc3bb2-724a-43ae-953f-651351a5a5f2,"Document: ---\ntitle: ""Baselines""\nslug: ""f...","Yes, you can have multiple baselines in Fiddle...",Can I have multiple baselines?,Great,1.0,2023-10-12 13:43:17.893
8,916d2a3e-42ef-46a2-8f61-982f59948e81,None,"Document: ---\ntitle: ""About the Fiddler Cl...",The purpose of Fiddler is to provide a platfor...,Why is Fiddler?,None,1.0,2023-10-12 13:46:49.122
9,7074d0ee-d66e-40e3-a2b8-a86292ae4180,b36feb54-7464-482c-ad6c-71edd91d1585,"Document: ---\ntitle: ""ML Algorithms In Fid...","Yes, Fiddler supports LLMs (Large Language Mod...",Can Fiddler support LLMs?,None,NaN,2023-10-12 14:52:07.649


In [16]:
#session.execute("DELETE FROM fiddler_chatbot_conversation WHERE row_id='-1'")

In [35]:
#session.execute("DROP TABLE fiddler_chatbot_history")

In [23]:
#session.execute("TRUNCATE TABLE fiddler_doc_snippets_openai")

In [27]:
session.execute("TRUNCATE TABLE fiddler_chatbot_history")